## Overview ##

Based on Starter Kernel [Logistic Regression on Tournament Seeds by Kasper P. Lauritzen](https://www.kaggle.com/kplauritzen/notebookde27b18258?scriptVersionId=804590) 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from subprocess import check_output
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix



/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the training data ##
We're keeping it simple & using only 2 files for this model: the Tourney seeds & Compact results.

In [133]:
data_dir = '../input/'
df_seeds = pd.read_csv(data_dir + 'NCAATourneySeeds.csv')
#df_tour = pd.read_csv(data_dir + 'NCAATourneyDetailedResults.csv')
df_tour = pd.read_csv(data_dir + 'RegularSeasonCompactResults.csv')
teams = pd.read_csv(data_dir + 'Teams.csv')

def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_tour.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], inplace=True, axis=1)
df_tour.head()


,Season,WTeamID,LTeamID
0,1985,1228,1328
1,1985,1106,1354
2,1985,1112,1223
3,1985,1165,1432
4,1985,1192,1447


Build out the id string (year_wteam_lteam)
Create a prediction at the end.

In [159]:
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tour, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_true=df_concat.copy()

columnsTitles=["LTeamID","WTeamID"]
df_false = df_concat.rename(columns={'WTeamID': 'LTeamID_', 'LTeamID': 'WTeamID_'}) #idk if i have to do this I just didnt want to create issues
df_false = df_false.rename(columns={'LTeamID_': 'LTeamID', 'WTeamID_': 'WTeamID'})
df_false = df_false.rename(columns={'WSeed': 'LSeed_', 'LSeed': 'WSeed_'}) #idk if i have to do this I just didnt want to create issues
df_false = df_false.rename(columns={'LSeed_': 'LSeed', 'WSeed_': 'WSeed'})


df_true['label'] = 1
df_false['label'] = 0

df = df_true.append(df_false, ignore_index=True)
df = df.fillna(17)

#df['WTID']=df.apply(build_wteamyear, axis=1) #again, probably better way to do this and save a function
#df['LTID']=df.apply(build_lteamyear, axis=1)
df['SeedDiff'] =  df.WSeed - df.LSeed #Lower seeds are better, so Negative means that the lower seed team won.
df = df[df['Season'] > 2013] # We want to train on 2014 and above.

df = df[['WTeamID', 'LTeamID', 'SeedDiff', 'label']]
df.head()



,WTeamID,LTeamID,SeedDiff,label
15289,1103,1157,1.0,1
15290,1279,1157,1.0,1
15291,1278,1157,1.0,1
15292,1418,1157,-2.0,1
15293,1155,1157,1.0,1


## Encode and, Scale and Split ##

In [160]:
X = df.iloc[:, 0:3].values
y = df.iloc[:, 3].values


enc_1 = LabelEncoder()
X[:, 0] = enc_1.fit_transform(X[:, 0])
enc_2 = LabelEncoder()
X[:, 1] = enc_2.fit_transform(X[:, 1])
onehotencoder = OneHotEncoder(categorical_features = [0,1],handle_unknown='error') #handle_unknown='unknown'or impute
X = onehotencoder.fit_transform(X).toarray()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

ds_width=X_test.shape[1]


## Build the classifier and hit the button ##

In [162]:
from keras.layers import LeakyReLU
from keras import metrics
from keras import optimizers

# 61%  //Dense(6, he_normal)//leakyrelu alpha 0.2//Dense(16, he_normal)
# leakyrelu alpha 0.2//Dense(16, he_normal)//leakyrelu alpha 0.2//Dense(1, he_normal, sigmoid)//SGD, Mean_Squared_Error
# Third Attempt.
classifier = Sequential()
classifier.add(Dense(13,input_shape=(ds_width,),kernel_initializer = 'normal',activation='relu'))
#classifier.add(LeakyReLU(alpha=0.2))
classifier.add(Dense(6, activation='relu',kernel_initializer = 'normal',))
#classifier.add(LeakyReLU(alpha=0.2))
#classifier.add(Dense(64, activation='relu',kernel_initializer = 'normal',))
classifier.add(Dense(1,kernel_initializer = 'normal' ))
classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['binary_accuracy']) #binary_crossentropy
print("Cooking... Please wait")
classifier.fit(X_train, y_train, batch_size = 250, epochs = 200, verbose=1)
print("*ding*... Testing Accuracy on test set")
#Run Predictions on the test set
pred_test = classifier.predict(X_test)

pred_test = (pred_test > 0.5)
cm = confusion_matrix(pred_test, y_test)
print ((cm[0][0]+cm[1][1])/(sum(cm[0])+sum(cm[1])))

Cooking... Please wait
Epoch 1/200
3891/3891 [==============================] - 3s 880us/step - loss: 0.4698 - binary_accuracy: 0.5009
Epoch 2/200
3891/3891 [==============================] - 0s 20us/step - loss: 0.4061 - binary_accuracy: 0.5009
Epoch 3/200
3891/3891 [==============================] - 0s 16us/step - loss: 0.3108 - binary_accuracy: 0.5163
Epoch 4/200
3891/3891 [==============================] - 0s 19us/step - loss: 0.2155 - binary_accuracy: 0.6279
Epoch 5/200
3891/3891 [==============================] - 0s 23us/step - loss: 0.1823 - binary_accuracy: 0.7137
Epoch 6/200
3891/3891 [==============================] - 0s 20us/step - loss: 0.1690 - binary_accuracy: 0.7422
Epoch 7/200
3891/3891 [==============================] - 0s 23us/step - loss: 0.1601 - binary_accuracy: 0.7538
Epoch 8/200
3891/3891 [==============================] - 0s 24us/step - loss: 0.1539 - binary_accuracy: 0.7646
Epoch 9/200
3891/3891 [==============================] - 0s 24us/step - loss: 0.1494 - b

3891/3891 [==============================] - 0s 25us/step - loss: 0.0877 - binary_accuracy: 0.8746
Epoch 74/200
3891/3891 [==============================] - 0s 19us/step - loss: 0.0876 - binary_accuracy: 0.8743
Epoch 75/200
3891/3891 [==============================] - 0s 19us/step - loss: 0.0869 - binary_accuracy: 0.8736
Epoch 76/200
3891/3891 [==============================] - 0s 20us/step - loss: 0.0867 - binary_accuracy: 0.8774
Epoch 77/200
3891/3891 [==============================] - 0s 25us/step - loss: 0.0865 - binary_accuracy: 0.8733
Epoch 78/200
3891/3891 [==============================] - 0s 24us/step - loss: 0.0863 - binary_accuracy: 0.8774
Epoch 79/200
3891/3891 [==============================] - 0s 25us/step - loss: 0.0866 - binary_accuracy: 0.8754
Epoch 80/200
3891/3891 [==============================] - 0s 26us/step - loss: 0.0860 - binary_accuracy: 0.8766
Epoch 81/200
3891/3891 [==============================] - 0s 22us/step - loss: 0.0855 - binary_accuracy: 0.8782
Epoch

3891/3891 [==============================] - 0s 22us/step - loss: 0.0766 - binary_accuracy: 0.8846
Epoch 146/200
3891/3891 [==============================] - 0s 18us/step - loss: 0.0774 - binary_accuracy: 0.8828
Epoch 147/200
3891/3891 [==============================] - 0s 22us/step - loss: 0.0770 - binary_accuracy: 0.8838
Epoch 148/200
3891/3891 [==============================] - 0s 16us/step - loss: 0.0772 - binary_accuracy: 0.8833
Epoch 149/200
3891/3891 [==============================] - 0s 20us/step - loss: 0.0768 - binary_accuracy: 0.8841
Epoch 150/200
3891/3891 [==============================] - 0s 24us/step - loss: 0.0766 - binary_accuracy: 0.8856
Epoch 151/200
3891/3891 [==============================] - 0s 27us/step - loss: 0.0770 - binary_accuracy: 0.8872
Epoch 152/200
3891/3891 [==============================] - 0s 23us/step - loss: 0.0764 - binary_accuracy: 0.8869
Epoch 153/200
3891/3891 [==============================] - 0s 21us/step - loss: 0.0766 - binary_accuracy: 0.88

## Make the prediction ##

In [ ]:
def format_dataset(df_sample):
    df_pred = pd.DataFrame()
    df_pred['Season'], df_pred['WTeamID'], df_pred['LTeamID'] = df_sample_sub['ID'].str.split('_', 2).str
    df_pred = df_pred.astype(int) #DO I NEED THIS?
    df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
    df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
    df_preddummy = pd.merge(left=df_pred, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
    df_predmerge = pd.merge(left=df_preddummy, right=df_lossseeds, on=['Season', 'LTeamID'])
    df_predmerge['SeedDiff'] = df_predmerge.WSeed - df_predmerge.LSeed
    df_predmerge = df_predmerge.rename(columns={'Season':'Year'})
    df_predictions = df_predmerge[['WTeamID','LTeamID','SeedDiff']] # We really should add year to this somewhere
    df_predictions['WTeamID'] = df_predictions['WTeamID'].astype(str)
    df_predictions['LTeamID'] = df_predictions['LTeamID'].astype(str)
    for x in df_predictions.columns:
        df_predictions[x]=df_predictions[x].astype(df_preenc[x].dtypes.name)
    return df_predictions.iloc[:, 0:3].values


#Convert the prediction table to our format
df_predX = format_dataset(pd.read_csv(data_dir + 'SampleSubmissionStage1.csv'))
#Encode the Predictions
df_predX[:, 0] = enc_1.transform(df_predX[:, 0])
df_predX[:, 1] = enc_2.transform(df_predX[:, 1])
df_predX = onehotencoder.transform(df_predX).toarray()
df_predX = sc.transform(df_predX)
#Generate the predictions and save
y_p = classifier.predict(df_predX)
clipped_preds = np.clip(y_p, 0.0001, 0.999)
df_out=df_sample_sub
df_out.Pred = clipped_preds
df_out.to_csv('out.csv', index=False)
display(df_out)
print(df_out['Pred'].std())
